In [1]:
!pip install transformers==4.4.2
!pip install sentencepiece==0.1.95
!pip install datasets==1.5.0
!pip install nlp==0.4.0
!pip install nltk
!python -m nltk.downloader punkt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 26.0 MB/s 
     |████████████████████████████████| 3.3 MB 44.9 MB/s 
     |████████████████████████████████| 880 kB 43.7 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=80399bcf0a27f366682a50f49c2d4649868db35164196c769c3d5af67f9fff13
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 16.5 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 192 kB 25.8 MB/s 
     |████████████████████████████████| 69 kB 8.2 MB/s 
     |████████████████████████████████| 140 kB 50.9 MB/s 
  

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd gdrive/My Drive/Colab Notebooks/QG_idt5_t5

Mounted at /content/gdrive
/content/gdrive/My Drive/Colab Notebooks/QG_idt5_t5


In [3]:
%cd multitask-question-generation

/content/gdrive/My Drive/Colab Notebooks/QG_idt5_t5/multitask-question-generation


Prepare Data

In [ ]:
!python prepare_data.py

Downloading: 100% 756k/756k [00:00<00:00, 6.51MB/s]
Downloading: 100% 65.0/65.0 [00:00<00:00, 50.3kB/s]
Downloading: 100% 173/173 [00:00<00:00, 132kB/s]
100% 65698/65698 [00:02<00:00, 30573.80it/s]
06/07/2022 06:24:11 - INFO - nlp.arrow_writer -   Done writing 65698 examples in 51501866 bytes .
100% 65698/65698 [00:02<00:00, 27268.16it/s]
06/07/2022 06:24:14 - INFO - nlp.arrow_writer -   Done writing 65698 examples in 50875718 bytes .
/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:175: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
100% 66/66 [01:56<00:00,  1.76s/it]
06/07/2022 06:26:10 - INFO - nlp.arrow_writer -   Done writing 65698 examples in 631909622 bytes .
100% 15881/15881 [00:00<00:00, 32311.61it/s]
06/07/2022 06:26:11 - INFO - nlp.arrow_wr

Fine Tune

In [4]:
!python3 run_multi.py \
    --model_name_or_path muchad/idt5-base \
    --model_type t5 \
    --tokenizer_name_or_path t5_qg_tokenizer \
    --output_dir t5-base-multi \
    --train_file_path data/train_data_multitask_t5.pt \
    --valid_file_path data/valid_data_multitask_t5.pt \
    --per_device_train_batch_size 2 \
    --per_device_eval_batch_size 2 \
    --gradient_accumulation_steps 2 \
    --learning_rate 1e-4 \
    --num_train_epochs 1 \
    --seed 42 \
    --do_train \
    --do_eval \
    --logging_steps 100 \
    --prediction_loss_only True

06/07/2022 06:50:02 - WARNING - __main__ -   Process rank: -1, device: cuda:0, n_gpu: 1, distributed training: False, 16-bits training: False
06/07/2022 06:50:02 - INFO - __main__ -   Training/evaluation parameters TrainingArguments(output_dir=t5-base-multi, overwrite_output_dir=False, do_train=True, do_eval=True, do_predict=False, evaluation_strategy=IntervalStrategy.NO, prediction_loss_only=True, per_device_train_batch_size=2, per_device_eval_batch_size=2, gradient_accumulation_steps=2, eval_accumulation_steps=None, learning_rate=0.0001, weight_decay=0.0, adam_beta1=0.9, adam_beta2=0.999, adam_epsilon=1e-08, max_grad_norm=1.0, num_train_epochs=1.0, max_steps=-1, lr_scheduler_type=SchedulerType.LINEAR, warmup_ratio=0.0, warmup_steps=0, logging_dir=runs/Jun07_06-50-02_27efe21eebd8, logging_strategy=IntervalStrategy.STEPS, logging_first_step=False, logging_steps=100, save_strategy=IntervalStrategy.STEPS, save_steps=500, save_total_limit=None, no_cuda=False, seed=42, fp16=False, fp16_opt

In [5]:
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
from pipelines import pipeline
pipe = pipeline("multitask-qa-qg", model = "t5-base-multi")

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [7]:
text1 = "Pangeran Harya Dipanegara (atau biasa dikenal dengan nama Pangeran Diponegoro, lahir di Ngayogyakarta Hadiningrat, 11 November 1785 – meninggal di Makassar, Hindia Belanda, 8 Januari 1855 pada umur 69 tahun) adalah salah seorang pahlawan nasional Republik Indonesia, yang memimpin Perang Diponegoro atau Perang Jawa selama periode tahun 1825 hingga 1830 melawan pemerintah Hindia Belanda. Sejarah mencatat, Perang Diponegoro atau Perang Jawa dikenal sebagai perang yang menelan korban terbanyak dalam sejarah Indonesia, yakni 8.000 korban serdadu Hindia Belanda, 7.000 pribumi, dan 200 ribu orang Jawa serta kerugian materi 25 juta Gulden."

In [8]:
pipe(text1)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1764: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


[{'answer': 'Pangeran Harya Dipanegara',
  'question': 'Siapa yang memimpin Perang Diponegoro atau Perang Jawa selama periode tahun 1825 hingga 1830?'},
 {'answer': '25 juta',
  'question': 'Berapa banyak orang yang kerugian dari Perang Diponegoro atau Perang Jawa?'}]

In [9]:
text2 = "Pemerintah berencana menaikkan tiket Candi Borobudur menjadi Rp 750.000/orang. Kenaikan harga tiket ini bertujuan untuk membatasi jumlah pengunjung sehingga kondisi candi sebagai cagar budaya tetap ini tetap lestari. Namun, harga tiket yang mencapai Rp 750.000 per orang itu bukanlah tiket masuk, melainkan tiket naik ke atas Candi Borobudur. Direktur Utama PT Aviasi Pariwisata Indonesia (Persero)/InJourney, Dony Oskaria menyebut tiket masuk Candi Borobudur tak berubah."
pipe(text2)

/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1764: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  next_indices = next_tokens // vocab_size


[{'answer': '750.000/orang',
  'question': 'Berapa harga tiket masuk ke Candi Borobudur?'},
 {'answer': 'cagar budaya',
  'question': 'Selain tiket masuk, kondisi apa yang tetap tetap lestari?'},
 {'answer': 'tiket masuk',
  'question': 'Apa yang dimaksud dengan tiket naik ke atas Candi Borobudur?'},
 {'answer': 'Dony Oskaria',
  'question': 'Siapa yang menyebut tiket masuk Candi Borobudur tak berubah?'}]

In [10]:
pipe({"context":text1,"question":"kapan pangeran diponegoro lahir?"})

'11 November 1785'

In [11]:
pipe({"context":text1,"question":"kapan pangeran diponegoro meninggal?"})

'8 Januari 1855'

In [12]:
pipe({"context":text2,"question":"berapa harga tiket candi borobudur?"})

'750.000'

In [13]:
pipe({"context":text2,"question":"siapa itu dony oskaria?"})

'Direktur Utama'

In [14]:
pipe({"context":text2,"question":"apa tujuan kenaikan harga tiket?"})

'membatasi jumlah pengunjung sehingga kondisi candi sebagai cagar budaya tetap ini tetap lestari'

In [15]:
pipe({"context":text2,"question":"siapa nama direktur?"})

'Dony Oskaria'